In [1]:
import fastavro
import requests
import io
import pandas as pd

In [2]:
# oids = pd.read_csv("sent.txt",names=["oid"])

# List of SN candidates

In [3]:
refstring = "20190901"
oids = open("%s.txt" % (refstring)).read().splitlines()
print(oids)

['ZTF19aabhejl', 'ZTF19abvgxrq', 'ZTF19abvgxoq', 'ZTF19abvgxde', 'ZTF19abvgwcs', 'ZTF19abvfvrq', 'ZTF19abvehtp', 'ZTF19abvdgqo', 'ZTF19abvakbo', 'ZTF19abvdbyx', 'ZTF19abvalfd', 'ZTF19abvanim']


In [4]:
oids = pd.DataFrame(oids)
oids.columns = ["oid"]
oids

,oid
0,ZTF19aabhejl
1,ZTF19abvgxrq
2,ZTF19abvgxoq
3,ZTF19abvgxde
4,ZTF19abvgwcs
5,ZTF19abvfvrq
6,ZTF19abvehtp
7,ZTF19abvdgqo
8,ZTF19abvakbo
9,ZTF19abvdbyx


# Getting discovery avro

In [5]:
all_data = []
not_processed = []
for i,oid in oids.iterrows():
    print(oid)
    url = "http://ztf.alerce.online/get_detections"
    resp = requests.post(url, json={"oid":oid.oid})
    df = pd.DataFrame(resp.json()["result"]["detections"]).sort_values("mjd")
    first_detect = df.iloc[0].candid_str
    try:
        response = requests.get("http://avro.alerce.online/get_avro?oid={}&candid={}".format(oid,first_detect))
        avro = io.BytesIO(response.content)
        reader = fastavro.reader(avro)
        data = reader.next()
        all_data.append({"oid":oid.oid,"ssdistnr": data["candidate"]["ssdistnr"],"ssmagnr": data["candidate"]["ssmagnr"],"ssnamenr": data["candidate"]["ssnamenr"]})
    except:
        not_processed.append(oid.oid)

oid    ZTF19aabhejl
Name: 0, dtype: object
oid    ZTF19abvgxrq
Name: 1, dtype: object
oid    ZTF19abvgxoq
Name: 2, dtype: object
oid    ZTF19abvgxde
Name: 3, dtype: object
oid    ZTF19abvgwcs
Name: 4, dtype: object
oid    ZTF19abvfvrq
Name: 5, dtype: object
oid    ZTF19abvehtp
Name: 6, dtype: object
oid    ZTF19abvdgqo
Name: 7, dtype: object
oid    ZTF19abvakbo
Name: 8, dtype: object
oid    ZTF19abvdbyx
Name: 9, dtype: object
oid    ZTF19abvalfd
Name: 10, dtype: object
oid    ZTF19abvanim
Name: 11, dtype: object


In [6]:
all_data = pd.DataFrame(all_data)

# These are bad SN

In [7]:
all_data[all_data.ssdistnr != -999]

,oid,ssdistnr,ssmagnr,ssnamenr


# These are good SN

In [8]:
all_data[all_data.ssdistnr == -999]

,oid,ssdistnr,ssmagnr,ssnamenr
0,ZTF19aabhejl,-999.0,-999.0,null
1,ZTF19abvgxrq,-999.0,-999.0,null
2,ZTF19abvgxoq,-999.0,-999.0,null
3,ZTF19abvgxde,-999.0,-999.0,null
4,ZTF19abvgwcs,-999.0,-999.0,null
5,ZTF19abvfvrq,-999.0,-999.0,null
6,ZTF19abvehtp,-999.0,-999.0,null
7,ZTF19abvdgqo,-999.0,-999.0,null
8,ZTF19abvakbo,-999.0,-999.0,null
9,ZTF19abvdbyx,-999.0,-999.0,null


# Some exception was raised

In [19]:
not_processed

[]

# Change not_processed index to select the candidate

In [19]:
oid = not_processed[1]
url = "http://ztf.alerce.online/get_detections"
resp = requests.post(url, json={"oid":oid})
df = pd.DataFrame(resp.json()["result"]["detections"]).sort_values("mjd")
first_detect = df.iloc[0].candid_str

IndexError: list index out of range

In [ ]:
response = requests.get("http://avro.alerce.online/get_avro?oid={}&candid={}".format(oid,first_detect))

In [ ]:
avro = io.BytesIO(response.content)
reader = fastavro.reader(avro)
data = reader.next()

In [104]:
data["candidate"]["ssmagnr"]

18.100000381469727

In [105]:
data["candidate"]["ssnamenr"]

'266079'

In [106]:
oid

'ZTF19abmzlrt'